In [1]:
import datasets

dataset = datasets.load_dataset("nvm472001/cvdataset-layoutlmv3")

Found cached dataset cvdataset-layoutlmv3 (/home/lhnguyen/.var/app/com.jetbrains.PyCharm-Professional/cache/huggingface/datasets/nvm472001___cvdataset-layoutlmv3/CV Extractions/1.0.0/6f6f7a3774790bc48dc2f69c44ec9d6c84524adc9b71a61e73296ca98718b7bb)


  0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
train_set = dataset['train'][0]
train_set['image_path']

'/home/lhnguyen/.var/app/com.jetbrains.PyCharm-Professional/cache/huggingface/datasets/downloads/extracted/a58915a7257627075e25b8d892ac29156622f048ce31621cb031d60473a9f61c/data2/raw/img/imgCV_92.jpg'

In [3]:
words, bboxes, ner_tags = train_set["words"], train_set["bboxes"], train_set["ner_tags"]
print(words)
print(bboxes)
print(ner_tags)

['NGUYEN NGOC NGHIA ', 'ARTIFICIAL INTELLIGENCE ENGINEER ', 'OBJECTIVE ', 'am looking for a full time job in Artificial Intelligence with the objective is building a long-term ', 'career in Al field with opportunities for career growth. ', 'EDUCATION ', 'HO CHI MINH CITY UNIVERSITY OF TECHNOLOGY ', 'SEP 2014 - NOV 2019 ', 'Major: Mechatronics ', 'WORK EXPERIENCE ', '12/10/1996 ', 'Male ', 'FPT SOFTWARE COMPANY ', 'JAN 2020 - NOW ', '0339405675 ', 'Al engineer ', 'Understand and have the ability to redesign; implement new deep learning models. ', 'nnnghia.96@gmail.com ', 'Have ability to build full pipeline of a deep learning model. ', 'Hoa Vinh, Dong Hoa, Phu Yen ', 'Integrated weakly supervised training method to text detection model. ', 'Wrote evaluation for text detection and text recognition models. ', 'Wrote rotation model to detect orientation of imaae ', 'Used tablenet for table extraction:. ', 'SKILLS ', 'Used SynthText model to generate fake data for Optical Character Recognit

In [4]:
from transformers import AutoProcessor
processor =  AutoProcessor.from_pretrained("microsoft/layoutlmv3-large", apply_ocr=False)

In [5]:
from datasets.features import ClassLabel

features = dataset["train"].features
column_names = dataset["train"].column_names
image_column_name = "image_path"
text_column_name = "words"
boxes_column_name= "bboxes"
label_column_name = "ner_tags"

def get_label_list(labels):
  unique_labels = set()

  for label in labels:
    unique_labels = unique_labels | set(label)

  labels_list = list(unique_labels)
  labels_list.sort()

  return labels_list

if isinstance(features[label_column_name].feature, ClassLabel):
  label_list = features[label_column_name].feature.names
else:
  label_list = get_label_list(dataset["train"][label_column_name])
id2label = {k:v for k, v in enumerate(label_list)}
label2id = {v:k for k, v in enumerate(label_list)}
num_label = len(id2label)

In [6]:
print(num_label, label_list, id2label, sep="\n")

15
['person_name', 'dob_field', 'gender_field', 'phonenumber_field', 'email_field', 'address_field', 'socical_address_field', 'education', 'education_name', 'education_time', 'experience', 'experience_name', 'experience_time', 'information', 'undefined']
{0: 'person_name', 1: 'dob_field', 2: 'gender_field', 3: 'phonenumber_field', 4: 'email_field', 5: 'address_field', 6: 'socical_address_field', 7: 'education', 8: 'education_name', 9: 'education_time', 10: 'experience', 11: 'experience_name', 12: 'experience_time', 13: 'information', 14: 'undefined'}


In [7]:
from PIL import Image

def prepare_examples(examples):
  images = [Image.open(path).convert("RGB") for path in examples['image_path']] #Image.open(examples[image_column_name])
  words = examples[text_column_name]
  boxes = examples[boxes_column_name]
  word_labels = examples[label_column_name]

  encoding = processor(images, words, boxes=boxes, word_labels=word_labels,
                       truncation=True, padding="max_length")

  return encoding

In [8]:
from datasets import Features, Sequence, ClassLabel, Value, Array2D, Array3D

features = Features({
    'pixel_values': Array3D(dtype="float32", shape=(3, 224, 224)),
    'input_ids': Sequence(feature=Value(dtype='int64')),
    'attention_mask': Sequence(Value(dtype='int64')),
    'bbox': Array2D(dtype="int64", shape=(512, 4)),
    'labels': Sequence(feature=Value(dtype='int64')),
})

train_dataset = dataset["train"].map(
    prepare_examples,
    batched=True,
    remove_columns=column_names,
    features=features,
)
eval_dataset = dataset["test"].map(
    prepare_examples,
    batched=True,
    remove_columns=column_names,
    features=features,
)

Loading cached processed dataset at /home/lhnguyen/.var/app/com.jetbrains.PyCharm-Professional/cache/huggingface/datasets/nvm472001___cvdataset-layoutlmv3/CV Extractions/1.0.0/6f6f7a3774790bc48dc2f69c44ec9d6c84524adc9b71a61e73296ca98718b7bb/cache-54037c5842877dc3.arrow
Loading cached processed dataset at /home/lhnguyen/.var/app/com.jetbrains.PyCharm-Professional/cache/huggingface/datasets/nvm472001___cvdataset-layoutlmv3/CV Extractions/1.0.0/6f6f7a3774790bc48dc2f69c44ec9d6c84524adc9b71a61e73296ca98718b7bb/cache-c1f6d49f9b6051cb.arrow


In [9]:
train_dataset.set_format('torch')
train_dataset

Dataset({
    features: ['pixel_values', 'input_ids', 'attention_mask', 'bbox', 'labels'],
    num_rows: 220
})

In [10]:
example = train_dataset[0]
for k,v in example.items():
    print(k,v.shape)

pixel_values torch.Size([3, 224, 224])
input_ids torch.Size([512])
attention_mask torch.Size([512])
bbox torch.Size([512, 4])
labels torch.Size([512])


In [11]:
processor.tokenizer.decode(eval_dataset[0]["input_ids"])

'<s> PHAM DUC KIEN  UI UX DESIGNER  CAREER GOALS  Applying my own skills and experience to work; helping the  company increasingly develop.  Date of birth  01/05/2001  Accumulating more experience and developing myself in both  professional and soft skills.  Phone  0349625980  EDUCATION  Address  2019-now  University of Engineering and Technology  Cầu Giấy, Hà Nội  Being currently a junior in University of  Engineering and Technology.  E-mail  phamkien54362001 @gmail.com  12/2020-06/2021 Having graduated from Intensive Graphic  Design course at ColorMe  WORK EXPERIENCE  SOCIAL NETWORK  Viettel Post Joint Stock Corporation  Designer: responsible for designing media  Facebook  publications and developing webs for  Phạm Đức Kiên  ViettelPost and external customers.  Intagram  Horus Agency  phamduckien010501  Designer (remote): Designing logos.  Behance  Vipcom JSC  phamkien5436  Designer: Designing T-shirts to sell on Amazon  All time  Freelancer  SKILLS  Used to cooperate with several cl

In [12]:
from datasets import load_metric

metric = load_metric("seqeval")

<ipython-input-12-edef154daec7>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


In [13]:
import numpy as np

return_entity_level_metrics = False

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    if return_entity_level_metrics:
        # Unpack nested dictionaries
        final_results = {}
        for key, value in results.items():
            if isinstance(value, dict):
                for n, v in value.items():
                    final_results[f"{key}_{n}"] = v
            else:
                final_results[key] = value
        return final_results
    else:
        return {
            "precision": results["overall_precision"],
            "recall": results["overall_recall"],
            "f1": results["overall_f1"],
            "accuracy": results["overall_accuracy"],
        }

In [14]:
from transformers import LayoutLMv3ForTokenClassification

model = LayoutLMv3ForTokenClassification.from_pretrained("microsoft/layoutlmv3-large",
                                                         id2label=id2label,
                                                         label2id=label2id)

Some weights of LayoutLMv3ForTokenClassification were not initialized from the model checkpoint at microsoft/layoutlmv3-large and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
                    output_dir="../checkpoints/classification/layoutlmv3",
                    max_steps=2000,
                    per_device_train_batch_size=1,
                    per_device_eval_batch_size=1,
                    learning_rate=1e-5,
                    eval_steps=200,
                    evaluation_strategy="steps",
                    metric_for_best_model="f1"
                )

In [16]:
from transformers.data.data_collator import default_data_collator

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=processor,
    data_collator=default_data_collator,
    compute_metrics=compute_metrics,
)

max_steps is given, it will override any value given in num_train_epochs


In [17]:
import torch
torch.cuda.set_per_process_memory_fraction(0.5)

In [18]:
trainer.train()

/home/lhnguyen/.miniconda3/envs/image-proc/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 220
  Num Epochs = 10
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 2000
  Number of trainable parameters = 357070351


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 1.96 GiB total capacity; 1.34 GiB already allocated; 247.19 MiB free; 1001.12 MiB allowed; 1.34 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF